# Análisis Exploratorio de Datos (EDA)

Para comenzar con este eda, debemos haber previamente ejecutado el ETL sobre el dataset original de Yelp. Se explicará en la documentación como se creó el dataset a travez de todos los archivos originales, y como se han cruzado los ID.

1. Carga y limpieza inicial
En esta primera etapa, se cargará el archivo meta_df.parquet y se realizará una limpieza inicial del DataFrame. Eliminaremos las columnas que no son relevantes para nuestro análisis y filtraremos las que nos interesan, incluyendo información sobre reseñas, ubicaciones, categorías, calificaciones y otros detalles. Esto nos permitirá enfocarnos en los datos más útiles para nuestro cliente.

In [1]:
import pandas as pd

#cargar el archivo meta_df.parquet
df = pd.read_parquet('meta_df.parquet')

#columnas que consideramos utiles para el analisis
columnas_utiles = [
    'review_id', 'user_id', 'business_id', 'stars_x', 'attributes', 'useful', 'funny', 'cool',
    'text', 'date', 'city', 'state', 'latitude', 'longitude', 'categories', 'hours'
]

#filtrar las columnas que son utiles
df = df[columnas_utiles]

#mostrar las primeras filas para verificar
print("Primeras filas del DataFrame filtrado:")
print(df.head())

#informacion general del dataframe
print("\nInformación general después de la limpieza inicial:")
print(df.info())

Primeras filas del DataFrame filtrado:
                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  VJxlBnJmCDIy8DFG0kjSow  Iaee7y6zdSB3B-kRCo4z1w  XQfwVwDr-v0ZS3_CbbE5Xw   
2  S6pQZQocMB1WHMjTRbt77A  ejFxLGqQcWNLdNByJlIhnQ  XQfwVwDr-v0ZS3_CbbE5Xw   
3  WqgTKVqWVHDHjnjEsBvUgg  f7xa0p_1V9lx53iIGN5Sug  XQfwVwDr-v0ZS3_CbbE5Xw   
4  M0wzFFb7pefOPcxeRVbLag  dCooFVCk8M1nVaQqcfTL3Q  XQfwVwDr-v0ZS3_CbbE5Xw   

   stars_x                                         attributes  useful  funny  \
0        3  {"NoiseLevel": "u'average'", "HasTV": "False",...       0      0   
1        2  {"NoiseLevel": "u'average'", "HasTV": "False",...       0      0   
2        4  {"NoiseLevel": "u'average'", "HasTV": "False",...       2      0   
3        3  {"NoiseLevel": "u'average'", "HasTV": "False",...       0      0   
4        2  {"NoiseLevel": "u'average'", "HasTV": "False",...       0      0   

   cool          

2. Estados con más reseñas
En este paso, contaremos el número de reseñas por estado, las ordenaremos de mayor a menor y generaremos un gráfico para visualizar los estados con mayor actividad de reseñas. Este análisis nos ayudará a identificar los mercados más relevantes para la expansión de la cadena de restaurantes.
* Consideraremos los estados que incluyan una buena cantidad de reseñas, ignoraremos las que tienen un numero insignificativo de estas.

In [2]:
import plotly.express as px
#filtrar los datos excluyendo registros con "valor incompleto" en el estado
df_filtrado = df[df['state'] != 'valor incompleto']

#contar el numero de resenas por estado
reseñas_por_estado = df_filtrado['state'].value_counts().reset_index()
reseñas_por_estado.columns = ['state', 'review_count']

#ordenar de mayor a menor
reseñas_por_estado = reseñas_por_estado.sort_values(by='review_count', ascending=False)

#mostrar los estados con mayor numero de resenas
print("Estados con más reseñas:")
print(reseñas_por_estado.head(10))

#filtrar los estados con al menos 10,000 resenas
reseñas_significativas = reseñas_por_estado[reseñas_por_estado['review_count'] >= 10000]

#graficar solo los estados con resenas significativas
fig = px.bar(reseñas_significativas, x='state', y='review_count', title='Estados con Mayor Número de Reseñas', labels={'review_count': 'Número de Reseñas', 'state': 'Estado'})
fig.show()


Estados con más reseñas:
  state  review_count
0    PA       1151002
1    FL        885484
2    TN        406893
3    MO        365908
4    IN        356811
5    LA        341859
6    AZ        340316
7    NJ        295037
8    NV        270132
9    AB        188924


3. En este análisis, identificaremos las categorías de negocios con más reseñas, lo que puede indicar su popularidad y relevancia en el mercado. Filtraremos categorías relacionadas con la industria de alimentos y restaurantes para proporcionar información valiosa para el cliente.

In [3]:
#filtrar y contar las categorias mas comunes
#expandir la lista de categorias en filas individuales
categorias_exploded = df.explode('categories')

#filtrar categorias que no sean 'valor incompleto'
categorias_filtradas = categorias_exploded[categorias_exploded['categories'] != 'valor incompleto']

#definir una lista de categorias relevantes relacionadas con la industria de alimentos y restaurantes
categorias_relevantes = [
    'Restaurants', 'Food', 'American (Traditional)', 'American (New)',
    'Breakfast & Brunch', 'Sandwiches', 'Seafood', 'Pizza', 'Coffee & Tea',
    'Italian', 'Burgers', 'Mexican', 'Salad', 'Cocktail Bars'
]

#filtrar el dataframe para conservar solo las categorias relevantes
categorias_filtradas = categorias_filtradas[categorias_filtradas['categories'].isin(categorias_relevantes)]

#contar la cantidad de resenas por categoria
reseñas_por_categoria = categorias_filtradas.groupby('categories').size().reset_index(name='review_count')

#ordenar de mayor a menor
reseñas_por_categoria = reseñas_por_categoria.sort_values(by='review_count', ascending=False)

#mostrar las categorias mas populares (maximo 20)
print("Categorías con más reseñas:")
#print(resenas_por_categoria.head(20))

#graficar las categorias con mas resenas
top_categorias = reseñas_por_categoria.head(20)
fig = px.bar(top_categorias, x='categories', y='review_count', title='Categorías con Mayor Número de Reseñas (Relevantes)', labels={'review_count': 'Número de Reseñas', 'categories': 'Categoría'})
fig.show()


Categorías con más reseñas:


4. Análisis de Frecuencia de Palabras en las Reseñas
Para entender mejor las reseñas de los clientes, realizamos un análisis de frecuencia de palabras en el texto de las reseñas. Hemos limpiado el texto para eliminar caracteres especiales y convertimos las palabras a minúsculas. Utilizamos CountVectorizer para identificar las palabras más comunes, excluyendo las palabras comunes del idioma inglés (stop words). Esto nos permite centrarnos en los términos que pueden ser más relevantes para analizar la satisfacción del cliente y sus principales preocupaciones.

In [4]:
#analisis de frecuencia de palabras en las resenas

from sklearn.feature_extraction.text import CountVectorizer
import re

#eliminar caracteres especiales y convertir el texto a minusculas
df['text_clean'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)).lower())

#configurar el countvectorizer para excluir palabras comunes en ingles (stop words)
vectorizer = CountVectorizer(stop_words='english', max_features=50)

#ajustar el vectorizer al texto limpio
X = vectorizer.fit_transform(df['text_clean'])

#obtener las palabras mas frecuentes y sus recuentos
frequencies = zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0))
sorted_frequencies = sorted(frequencies, key=lambda x: x[1], reverse=True)

#mostrar las 20 palabras mas comunes
print("Palabras más comunes en las reseñas:")
for word, count in sorted_frequencies[:20]:
    print(f"{word}: {count}")

#graficar las palabras mas comunes
top_words = sorted_frequencies[:20]
words, counts = zip(*top_words)
fig = px.bar(x=words, y=counts, title='Palabras Más Comunes en las Reseñas', labels={'x': 'Palabras', 'y': 'Frecuencia'})
fig.show()


Palabras más comunes en las reseñas:
food: 2704176
good: 2649062
place: 2471876
great: 2272975
just: 1792952
like: 1771210
time: 1693208
service: 1648035
really: 1313537
got: 1037602
nice: 971811
dont: 958351
im: 903784
ordered: 900239
best: 890305
ive: 873296
order: 863527
love: 834292
staff: 819090
did: 790327


### Interpretación de resultados:

* "food" (2.7M veces mencionada): La comida es claramente el tema principal en las reseñas, lo cual es esperado para un restaurante. Es importante analizar el contexto en el que se menciona "food" (positivo o negativo) para entender si los clientes están satisfechos con la calidad de los platos.

* "good" (2.6M veces) y "great" (2.3M veces): Estas palabras sugieren que muchos clientes tienen experiencias positivas en los establecimientos. Se podría profundizar en las frases que rodean estas palabras para conocer qué aspectos específicos (calidad de la comida, servicio, ambiente) se consideran "buenos" o "excelentes".

* "place" (2.4M veces): Los clientes suelen referirse al restaurante en general. Esta palabra, combinada con términos como "nice", "great", o "location", puede ayudar a identificar si la ubicación y el ambiente del restaurante son atractivos para los clientes.

* "service" (1.6M veces): El servicio es un factor crucial para la satisfacción del cliente. Es importante analizar el contexto en el que se menciona "service" para detectar si las menciones son positivas (e.g., "great service") o negativas (e.g., "poor service").

* "ordered", "order" (900K+ veces): Las palabras relacionadas con los pedidos sugieren que los clientes están comentando sobre su experiencia con el proceso de orden. Esto puede incluir la precisión de los pedidos, la rapidez o problemas con la comida solicitada.

* "staff" (819K veces): Los empleados son un componente clave en la experiencia del cliente. Evaluar si las menciones son positivas (e.g., "friendly staff") o negativas (e.g., "rude staff") ayudará a identificar áreas de mejora en el servicio.

* "love", "best", "really": Estas palabras indican fuertes sentimientos positivos, lo que es una buena señal. Es útil revisar las frases completas para entender qué aspectos generan tanto entusiasmo.

___

5. Análisis de Sentimientos en las Reseñas

Para identificar la actitud general en las reseñas, se realizó un análisis de sentimientos utilizando la librería `TextBlob`, la cual evalúa la polaridad de cada comentario:

- **Positivo:** Polaridad mayor a 0.2.
- **Negativo:** Polaridad menor a -0.2.
- **Neutral:** Polaridad entre -0.2 y 0.2.

El objetivo es categorizar las reseñas en estos tres grupos para entender mejor la percepción de los usuarios.

El código utilizado para clasificar los sentimientos y graficar la distribución es el siguiente:


In [5]:
import pandas as pd
from textblob import TextBlob
import plotly.express as px

#tomar una muestra aleatoria del 30% del dataframe
df_muestra = df.sample(frac=0.3, random_state=42)

#funcion para clasificar el sentimiento basado en la polaridad
def clasificar_sentimiento(texto):
    polaridad = TextBlob(texto).sentiment.polarity
    if polaridad > 0.2:
        return 'positivo'
    elif polaridad < -0.2:
        return 'negativo'
    else:
        return 'neutral'

#aplicar la funcion de clasificacion de sentimientos a la muestra
df_muestra['sentimiento'] = df_muestra['text'].apply(clasificar_sentimiento)

#verificar la cantidad de resenas positivas, negativas y neutrales
sentimientos_contados = df_muestra['sentimiento'].value_counts()
print("Distribución de Sentimientos en las Reseñas:")
print(sentimientos_contados)

#graficar la distribucion de sentimientos
fig = px.bar(sentimientos_contados, x=sentimientos_contados.index, y=sentimientos_contados.values,
             title='Distribución de Sentimientos en las Reseñas (Muestra de un %)',
             labels={'x': 'Sentimiento', 'y': 'Cantidad de Reseñas'})
fig.show()


Distribución de Sentimientos en las Reseñas:
sentimiento
positivo    902368
neutral     581130
negativo     41295
Name: count, dtype: int64


6. Análisis de Categorías según el Sentimiento de las Reseñas

En esta sección, analizaremos la relación entre el tipo de sentimiento en las reseñas (positivo, neutral, negativo) y las categorías de los negocios. Este análisis nos permitirá identificar cuáles son las categorías que tienden a recibir mejores o peores reseñas, lo cual puede ser útil para priorizar estrategias de marketing o tomar decisiones operativas.

El análisis consiste en:
- Filtrar los datos por el tipo de sentimiento (positivo, neutral, negativo).
- Contar la cantidad de negocios para cada categoría en cada grupo de sentimiento.
- Visualizar las 10 categorías más comunes en función del sentimiento, con el fin de identificar patrones de satisfacción o insatisfacción entre los clientes.


In [6]:
import pandas as pd
import plotly.express as px

#filtrar los datos por tipo de sentimiento
df_positivo = df_muestra[df_muestra['sentimiento'] == 'positivo']
df_neutral = df_muestra[df_muestra['sentimiento'] == 'neutral']
df_negativo = df_muestra[df_muestra['sentimiento'] == 'negativo']

#analisis de categorias
#contar la cantidad de negocios por categoria y sentimiento
def contar_categorias(df, tipo_sentimiento):
    categorias_exploded = df.explode('categories')
    categorias_contadas = categorias_exploded['categories'].value_counts().reset_index()
    categorias_contadas.columns = ['category', 'count']
    categorias_contadas['sentimiento'] = tipo_sentimiento
    return categorias_contadas

categorias_positivas = contar_categorias(df_positivo, 'positivo')
categorias_neutrales = contar_categorias(df_neutral, 'neutral')
categorias_negativas = contar_categorias(df_negativo, 'negativo')

#combinar los resultados en un solo dataframe
categorias_sentimiento = pd.concat([categorias_positivas, categorias_neutrales, categorias_negativas])

#graficar las 10 categorias mas comunes por sentimiento
fig = px.bar(categorias_sentimiento[categorias_sentimiento['category'].isin(
    categorias_sentimiento.groupby('category')['count'].sum().nlargest(10).index)],
    x='category', y='count', color='sentimiento', 
    title='Top 10 Categorías según el Sentimiento de las Reseñas',
    labels={'count': 'Cantidad de Negocios', 'category': 'Categoría'},
    barmode='group')
fig.show()


7. Análisis de la Distribución Geográfica de las Reseñas según el Sentimiento

Este análisis busca entender la distribución geográfica de las reseñas, considerando el tipo de sentimiento (positivo, neutral, negativo) en cada estado. Con esto, podemos identificar si existen diferencias significativas en la satisfacción de los clientes dependiendo de la ubicación geográfica.

Los pasos para este análisis son:
- Filtrar los datos por el tipo de sentimiento para agrupar las reseñas en positivo, neutral y negativo.
- Contar la cantidad de reseñas por estado en cada grupo de sentimiento.
- Visualizar los resultados para analizar la variación en la cantidad de reseñas y sentimientos a lo largo de diferentes estados.


In [7]:
#analisis de distribucion geografica
#contar la cantidad de resenas por estado y sentimiento
def contar_por_estado(df, tipo_sentimiento):
    estados_contados = df['state'].value_counts().reset_index()
    estados_contados.columns = ['state', 'count']
    estados_contados['sentimiento'] = tipo_sentimiento
    return estados_contados

estados_positivos = contar_por_estado(df_positivo, 'positivo')
estados_neutrales = contar_por_estado(df_neutral, 'neutral')
estados_negativos = contar_por_estado(df_negativo, 'negativo')

#combinar los resultados en un solo dataframe
estados_sentimiento = pd.concat([estados_positivos, estados_neutrales, estados_negativos])

#graficar la distribucion de resenas por estado y sentimiento
fig = px.bar(estados_sentimiento, x='state', y='count', color='sentimiento',
             title='Distribución de Reseñas por Estado y Sentimiento',
             labels={'count': 'Cantidad de Reseñas', 'state': 'Estado'},
             barmode='group')
fig.show()


8. Análisis de Servicios o Características y su Relación con el Sentimiento

En esta sección, exploraremos la relación entre los servicios o características de los negocios (representados en la columna `attributes`) y el sentimiento de las reseñas. La columna `attributes` contiene información valiosa sobre las características de cada negocio, como si permiten mascotas, ofrecen Wi-Fi gratis, tienen música en vivo, entre otros.

Para este análisis:
- Expandiremos la columna `attributes` para convertir los datos en un formato manejable.
- Filtraremos los negocios en función del sentimiento de las reseñas (positivo, neutral y negativo).
- Contaremos la frecuencia de los atributos en cada grupo de sentimiento.
- Graficaremos los atributos más comunes para ver si algunos están asociados con reseñas más positivas o negativas.

El objetivo es identificar qué características o servicios están vinculados a una percepción más favorable de los negocios, lo que puede ayudar a la cadena de restaurantes a decidir qué servicios ofrecer para mejorar la experiencia del cliente.


In [8]:
import ast

def expandir_atributos(df):
    #verificar que los valores sean validos y no 'valor incompleto'
    df['attributes'] = df['attributes'].apply(
        lambda x: ast.literal_eval(x) if pd.notnull(x) and isinstance(x, str) and x != 'valor incompleto' else {}
    )
    
    #expandir el diccionario de atributos en columnas separadas
    atributos_expandidos = df['attributes'].apply(pd.Series)
    df_expanded = pd.concat([df, atributos_expandidos], axis=1)
    return df_expanded

#expandir los atributos en el dataframe original
df_expanded = expandir_atributos(df_muestra)

#continuar con el analisis segun el sentimiento
df_positivo = df_expanded[df_expanded['sentimiento'] == 'positivo']
df_neutral = df_expanded[df_expanded['sentimiento'] == 'neutral']
df_negativo = df_expanded[df_expanded['sentimiento'] == 'negativo']


9. Análisis de Estados con Menciones de Palabras Clave Relacionadas con Mariscos y Pescados

En este análisis, identificamos las reseñas que mencionan palabras clave relevantes para el cliente, como "seafood" y "fish". Filtramos las reseñas para verificar si contienen estas palabras y luego analizamos la distribución de menciones por estado. Esto nos permite identificar las regiones con mayor interés en restaurantes de mariscos y pescados, ayudando al cliente a tomar decisiones informadas sobre dónde expandir su negocio.

In [9]:
import pandas as pd
import plotly.express as px

#palabras clave relacionadas con mariscos y pescados
palabras_clave = ['seafood', 'fish']

#filtrar resenas que contengan las palabras clave
def contiene_palabras_clave(texto, palabras_clave):
    texto = texto.lower()  #convertir el texto a minusculas
    return any(palabra in texto for palabra in palabras_clave)

df_muestra['contiene_palabra_clave'] = df_muestra['text'].apply(lambda x: contiene_palabras_clave(x, palabras_clave))

#contar las resenas con palabras clave por estado
reseñas_por_estado = df_muestra[df_muestra['contiene_palabra_clave']].groupby('state').size().reset_index(name='review_count')

#ordenar de mayor a menor
reseñas_por_estado = reseñas_por_estado.sort_values(by='review_count', ascending=False)

#seleccionar el top 10 de estados
top_estados = reseñas_por_estado.head(10)

#graficar los resultados
fig = px.bar(top_estados, x='state', y='review_count',
             title='Top 10 Estados con más Menciones de Palabras Relacionadas con Mariscos y Pescados',
             labels={'review_count': 'Cantidad de Reseñas', 'state': 'Estado'})
fig.show()


10. Análisis de la Distribución Temporal de las Reseñas

Este análisis examina cuándo los clientes tienden a dejar reseñas en función del día de la semana y la hora del día. Al agregar columnas que representan el día y la hora, podemos identificar los patrones temporales de actividad en las reseñas. Esto proporciona información sobre los momentos de mayor interacción con los clientes, lo que puede ser útil para optimizar estrategias de marketing o gestión de la operación en función de los días y horarios con mayor actividad.

In [10]:
import pandas as pd
import plotly.express as px

#asegurarse de que la columna 'date' este en formato de fecha
df_muestra['date'] = pd.to_datetime(df_muestra['date'])

#crear nuevas columnas para el dia de la semana y la hora del dia
df_muestra['day_of_week'] = df_muestra['date'].dt.day_name()
df_muestra['hour_of_day'] = df_muestra['date'].dt.hour

#contar la cantidad de resenas por dia de la semana
resenas_por_dia = df_muestra['day_of_week'].value_counts().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
).reset_index()
resenas_por_dia.columns = ['day_of_week', 'count']

#graficar la distribucion de resenas por dia de la semana
fig_dia = px.bar(resenas_por_dia, x='day_of_week', y='count',
                 title='Distribución de Reseñas por Día de la Semana',
                 labels={'count': 'Cantidad de Reseñas', 'day_of_week': 'Día de la Semana'})
fig_dia.show()

#contar la cantidad de resenas por hora del dia
resenas_por_hora = df_muestra['hour_of_day'].value_counts().sort_index().reset_index()
resenas_por_hora.columns = ['hour_of_day', 'count']

#graficar la distribucion de resenas por hora del dia
fig_hora = px.bar(resenas_por_hora, x='hour_of_day', y='count',
                  title='Distribución de Reseñas por Hora del Día',
                  labels={'count': 'Cantidad de Reseñas', 'hour_of_day': 'Hora del Día'})
fig_hora.show()


11. Análisis de Palabras Comunes en las Reseñas Positivas y Negativas

Este análisis identifica las palabras más comunes en las reseñas positivas y negativas, excluyendo palabras vacías (stopwords) que no aportan significado. Al filtrar y contar las palabras más frecuentes en cada tipo de sentimiento, podemos entender los aspectos más destacados tanto en las experiencias positivas como en las negativas de los clientes. Esto ayuda a resaltar los puntos fuertes y las áreas de mejora según la opinión de los usuarios.

In [11]:
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import plotly.express as px
import nltk

#descargar las stopwords en caso de no tenerlas
nltk.download('stopwords')

#lista de stopwords en ingles
stop_words = set(stopwords.words('english'))

#funcion para extraer palabras comunes de las resenas por sentimiento, excluyendo stopwords
def extraer_palabras_comunes(df, sentimiento, top_n=20):
    #filtrar las resenas segun el sentimiento
    df_sentimiento = df[df['sentimiento'] == sentimiento]
    
    #combinar todos los textos de resenas en una sola cadena
    texto_completo = ' '.join(df_sentimiento['text_clean'].dropna().values)
    
    #filtrar palabras que no sean stopwords
    palabras = [palabra for palabra in texto_completo.split() if palabra.lower() not in stop_words]
    
    #contar las palabras mas comunes
    palabras_comunes = Counter(palabras)
    
    #seleccionar las top_n palabras mas comunes
    palabras_comunes_df = pd.DataFrame(palabras_comunes.most_common(top_n), columns=['palabra', 'frecuencia'])
    palabras_comunes_df['sentimiento'] = sentimiento
    
    return palabras_comunes_df

#extraer palabras comunes para resenas positivas y negativas
palabras_positivas = extraer_palabras_comunes(df_muestra, 'positivo', top_n=20)
palabras_negativas = extraer_palabras_comunes(df_muestra, 'negativo', top_n=20)

#combinar los resultados en un solo dataframe
palabras_comunes = pd.concat([palabras_positivas, palabras_negativas])

#graficar las palabras mas comunes por sentimiento
fig_palabras = px.bar(palabras_comunes, x='palabra', y='frecuencia', color='sentimiento',
                      title='Aspectos Positivos y Negativos en las Reseñas',
                      labels={'frecuencia': 'Frecuencia de Palabras', 'palabra': 'Palabra'},
                      barmode='group')
fig_palabras.show()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mati\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
